In [1]:
import numpy as np
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import trange
import matplotlib.pyplot as plt
from natsort import natsorted

io.logger_setup() # run this to get printing of progress

model = models.CellposeModel(gpu=True)



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-10-22 22:00:56,491 [INFO] WRITING LOG OUTPUT TO C:\Users\lyh\.cellpose\run.log
2025-10-22 22:00:56,494 [INFO] 
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124
2025-10-22 22:00:56,598 [INFO] ** TORCH CUDA version installed and working. **
2025-10-22 22:00:56,599 [INFO] >>>> using GPU (CUDA)
2025-10-22 22:00:57,814 [INFO] >>>> loading model C:\Users\lyh\.cellpose\models\cpsam


#### Xenium FFPE Human Breast Cancer

In [ ]:
# Cellpose-SAM segmentation for Xenium FFPE Human Breast Cancer DAPI images
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()

# DAPI image path - Xenium FFPE Human Breast Cancer
dapi_path = "D:/segmentation_datasets/Xenium_FFPE_Human_Breast_Cancer_Rep1_outs/outs/morphology_mip.ome.tif"


# Load DAPI image
print(f"Loading DAPI image: {dapi_path}")
if not os.path.exists(dapi_path):
    print(f"Error: DAPI image file not found: {dapi_path}")
    exit()

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to new Cellpose format
# For DAPI single-channel image, we use the first channel
first_channel = '0'   # Use the first channel (DAPI)
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters - optimized for Xenium data
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 8           # Reduce batch size to fit large images

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

output_dir  = "cellpose_sam_segmentation"
# Save segmentation results
mask_filename = "cellpose_sam_masks_xenium_breast_cancer.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")


#### stereo seq sub image

In [ ]:
# Cellpose-SAM segmentation for Stereo-seq Mouse Brain data
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()

# Stereo-seq image path - Mouse Brain Adult
dapi_path = "D:/paper/newmodel/data/stereo_seq_mouse_brain/Mouse_brain_Adult_sub.tif"

# dapi_path = r"D:\paper\newmodel\data\NanoString_CosMx_Human_Pancreas\fov51\DAPI_F051.tif"

# Output directory for segmentation results
output_dir_base = r"D:/paper/newmodel/data/stereo_seq_mouse_brain/cellpose_sam_segmentation"

output_dir = os.path.join(output_dir_base, "cellpose_sam_segmentation")
os.makedirs(output_dir, exist_ok=True)

# Load Stereo-seq image
print(f"Loading Stereo-seq image: {dapi_path}")
if not os.path.exists(dapi_path):
    print(f"Error: Stereo-seq image file not found: {dapi_path}")
    exit()

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to new Cellpose format
# For Stereo-seq single-channel image, we use the first channel
first_channel = '0'   # Use the first channel
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters - optimized for Stereo-seq data
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 8           # Reduce batch size to fit large images

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

# Save segmentation results
mask_filename = "cellpose_sam_masks_stereo_seq_mouse_brain.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")


#### Xenium lung cancer


In [ ]:
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()


dapi_path = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_human_Lung_Cancer/dapi/dapi_channel.tif"

# Output directory for segmentation results
output_dir_base = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_human_Lung_Cancer_exp2/"
output_dir = os.path.join(output_dir_base, "cellpose_sam_segmentation")
os.makedirs(output_dir, exist_ok=True)

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to new Cellpose format
# For single-channel image, we use the first channel
first_channel = '0'   # Use the first channel
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters - optimized for Xenium data
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 8           # Reduce batch size to fit large images

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

# Save segmentation results
mask_filename = "cellpose_sam_masks_Xenium_human_Lung_Cancer.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")


creating new log file
2025-10-22 22:00:58,690 [INFO] WRITING LOG OUTPUT TO C:\Users\lyh\.cellpose\run.log
2025-10-22 22:00:58,691 [INFO] 
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124
原始图像形状: (17098, 51187)
原始图像数据类型: uint16, 最大值: 10866, 最小值: 0
转换后图像形状: (17098, 51187, 1)
图像是 uint16 类型，保持原始格式
选择的通道: [0]
处理后图像形状: (17098, 51187, 1)
初始化 CellposeModel (使用默认的 cpsam 模型)...
2025-10-22 22:01:00,881 [INFO] ** TORCH CUDA version installed and working. **
2025-10-22 22:01:00,881 [INFO] >>>> using GPU (CUDA)
2025-10-22 22:01:02,039 [INFO] >>>> loading model C:\Users\lyh\.cellpose\models\cpsam
运行 Cellpose-SAM 分割，参数如下:
- pretrained_model: cpsam (默认)
- flow_threshold: 0.4
- cellprob_threshold: 0.0
- tile_norm_blocksize: 0
- batch_size: 8
2025-10-22 22:18:01,591 [WARNING] WARNING: image is very large, not using gpu to compute flows from masks for QC step flow_threshold
2025-10-22 22:18:01,607 [INFO] turn off QC step with flow_threshold=0 if too 

#### Xenium_FFPE_Human_Pancreas

In [ ]:
 
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()


dapi_path = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_FFPE_Human_Pancreas/dapi/dapi_channel.tif"

# Output directory for segmentation results
output_dir_base = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_FFPE_Human_Pancreas_exp1"
output_dir = os.path.join(output_dir_base, "cellpose_sam_segmentation")
os.makedirs(output_dir, exist_ok=True)

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)

# Channel selection - according to new Cellpose format
# For single-channel image, we use the first channel
first_channel = '0'   # Use the first channel
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters - optimized for Xenium data
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 8           # Reduce batch size to fit large images

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

# Save segmentation results
mask_filename = "cellpose_sam_masks_Xenium_FFPE_Human_Pancreas.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")


creating new log file
2025-10-22 23:34:34,153 [INFO] WRITING LOG OUTPUT TO C:\Users\lyh\.cellpose\run.log
2025-10-22 23:34:34,153 [INFO] 
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124
原始图像形状: (13770, 34155)
原始图像数据类型: uint16, 最大值: 10002, 最小值: 0
转换后图像形状: (13770, 34155, 1)
图像是 uint16 类型，保持原始格式
选择的通道: [0]
处理后图像形状: (13770, 34155, 1)
初始化 CellposeModel (使用默认的 cpsam 模型)...
2025-10-22 23:34:35,629 [INFO] ** TORCH CUDA version installed and working. **
2025-10-22 23:34:35,629 [INFO] >>>> using GPU (CUDA)
2025-10-22 23:34:36,979 [INFO] >>>> loading model C:\Users\lyh\.cellpose\models\cpsam
运行 Cellpose-SAM 分割，参数如下:
- pretrained_model: cpsam (默认)
- flow_threshold: 0.4
- cellprob_threshold: 0.0
- tile_norm_blocksize: 0
- batch_size: 8
2025-10-23 00:06:37,423 [WARNING] more than 65535 masks in image, masks returned as np.uint32
分割完成。找到 136361 个细胞。
掩码文件已保存: C:/Users/lyh/paper/image_baseline_stitch/Xenium_FFPE_Human_Pancreas_exp1\cellpose_sam_s

#### Xenium_V1_mouse_Colon_FF

In [ ]:
 
import os
import numpy as np
import tifffile
import matplotlib.pyplot as plt
from cellpose import models, core, io, plot
from pathlib import Path
from tqdm import tqdm
from natsort import natsorted

# Set up logging
io.logger_setup()


dapi_path = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_V1_mouse_Colon_FF/dapi/dapi_channel.tif"

# Output directory for segmentation results
output_dir_base = "C:/Users/lyh/paper/image_baseline_stitch/Xenium_V1_mouse_Colon_FF_exp1"
output_dir = os.path.join(output_dir_base, "cellpose_sam_segmentation")
os.makedirs(output_dir, exist_ok=True)

dapi_img = tifffile.imread(dapi_path)
print(f"Original image shape: {dapi_img.shape}")
print(f"Original image dtype: {dapi_img.dtype}, max: {dapi_img.max()}, min: {dapi_img.min()}")

# Image preprocessing - handle possible multi-dimensional images
if len(dapi_img.shape) == 3:
    # If 3D image (possibly Z, Y, X or Y, X, C)
    if dapi_img.shape[0] < dapi_img.shape[1]:  # Channel might be in the first dimension
        # Assume (C, H, W) format, convert to (H, W, C)
        dapi_img = np.transpose(dapi_img, (1, 2, 0))
        print(f"Converted image format to (H, W, C): {dapi_img.shape}")
    # If multiple Z layers, take maximum projection or middle layer
    if dapi_img.shape[-1] > 3:  # If too many channels, might be Z layers
        print("Detected multi-layer image, using maximum intensity projection")
        dapi_img = np.max(dapi_img, axis=-1, keepdims=True)
elif len(dapi_img.shape) == 2:
    # If 2D image, convert to 3D (H, W, C)
    dapi_img = np.expand_dims(dapi_img, axis=-1)
    print(f"Converted image shape: {dapi_img.shape}")

# Image normalization
if dapi_img.max() > 0:
    if dapi_img.dtype == np.uint16:
        print("Image is uint16 type, keeping original format")
        dapi_img_norm = dapi_img.copy()
    elif dapi_img.max() > 255:
        print("Normalizing image to 0-255 (uint8)")
        dapi_img_norm = (dapi_img / dapi_img.max() * 255).astype(np.uint8)
    else:
        print("Image is already in suitable intensity range")
        dapi_img_norm = dapi_img.astype(np.uint8)
else:
    print("Warning: Image max value is 0, might be empty image")
    dapi_img_norm = dapi_img.astype(np.uint8)


# Channel selection - according to new Cellpose format
first_channel = '0'   # Use the first channel
second_channel = 'None'  # No second channel
third_channel = 'None'   # No third channel

selected_channels = []
for i, c in enumerate([first_channel, second_channel, third_channel]):
    if c == 'None':
        continue
    if int(c) >= dapi_img_norm.shape[-1]:  # Fixed boundary check
        print(f"Warning: Channel index {c} exceeds image channel count {dapi_img_norm.shape[-1]}")
        continue
    if c != 'None':
        selected_channels.append(int(c))

print(f"Selected channels: {selected_channels}")

# Create image with selected channels
img_selected_channels = np.zeros_like(dapi_img_norm)
if len(selected_channels) > 0:
    img_selected_channels[:, :, :len(selected_channels)] = dapi_img_norm[:, :, selected_channels]
else:
    # If no channels selected, use original image
    img_selected_channels = dapi_img_norm

print(f"Processed image shape: {img_selected_channels.shape}")

# Initialize new version of CellposeModel (default using cpsam model)
print("Initializing CellposeModel (using default cpsam model)...")
model = models.CellposeModel(gpu=True)  # Default pretrained_model="cpsam"

# Set segmentation parameters
flow_threshold = 0.4     # Flow threshold
cellprob_threshold = 0.0 # Cell probability threshold
tile_norm_blocksize = 0  # Tile normalization block size, 0 means no tile normalization
batch_size = 2           # Reduce batch size to fit large images

print(f"Running Cellpose-SAM segmentation with the following parameters:")
print(f"- pretrained_model: cpsam (default)")
print(f"- flow_threshold: {flow_threshold}")
print(f"- cellprob_threshold: {cellprob_threshold}")
print(f"- tile_norm_blocksize: {tile_norm_blocksize}")
print(f"- batch_size: {batch_size}")

# Run new version of Cellpose-SAM segmentation
masks, flows, styles = model.eval(
    img_selected_channels, 
    batch_size=batch_size, 
    flow_threshold=flow_threshold, 
    cellprob_threshold=cellprob_threshold,
    normalize={"tile_norm_blocksize": tile_norm_blocksize}
)

print(f"Segmentation complete. Found {masks.max()} cells.")

# Save segmentation results
mask_filename = "cellpose_sam_masks_Xenium_V1_mouse_Colon_FF.tif"
mask_path = os.path.join(output_dir, mask_filename)
tifffile.imwrite(mask_path, masks.astype(np.uint32))
print(f"Mask file saved: {mask_path}")




Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 


2025-10-23 22:25:31,640 [INFO] WRITING LOG OUTPUT TO C:\Users\lyh\.cellpose\run.log
2025-10-23 22:25:31,640 [INFO] 
cellpose version: 	4.0.4 
platform:       	win32 
python version: 	3.10.18 
torch version:  	2.5.0+cu124
原始图像形状: (34104, 28482)
原始图像数据类型: uint16, 最大值: 10576, 最小值: 0
转换后图像形状: (34104, 28482, 1)
图像是 uint16 类型，保持原始格式
选择的通道: [0]
处理后图像形状: (34104, 28482, 1)
初始化 CellposeModel (使用默认的 cpsam 模型)...
2025-10-23 22:25:34,236 [INFO] ** TORCH CUDA version installed and working. **
2025-10-23 22:25:34,237 [INFO] >>>> using GPU (CUDA)
2025-10-23 22:25:35,393 [INFO] >>>> loading model C:\Users\lyh\.cellpose\models\cpsam
运行 Cellpose-SAM 分割，参数如下:
- pretrained_model: cpsam (默认)
- flow_threshold: 0.4